### Objective : Find N Low Price to Book value stocks in KOSPI200
### Result: Stock list

| date       	| code    	| weight 	|
|------------	|---------	|--------	|
| 2010-01-31 	| A005930 	| 0.3333 	|
| 2010-01-31 	| A000010 	| 0.3333 	|
| 2010-01-31 	| A033230 	| 0.3333 	|
| 2010-04-30 	| A005930 	| 0.5    	|
| 2010-04-30 	| A323280 	| 0.5    	|

### To-do

1. Load Dataset (PER data)
2. Make rebalancing schedule using some packages (datetime, calendar)
3. Get N Low PER stocks at "specific" rebalancing date
4. Create table
5. Expand it to whole rebalancing schedule

In [1]:
import numpy as np
import pandas as pd
import calendar

# Load PE ratio Data and K200 Data
raw_per = pd.read_excel('PE Ratio_190530.xlsx', sheet_name = 'PE')
in_k200 = pd.read_excel('PE Ratio_190530.xlsx', sheet_name = 'K200')

In [2]:
raw_per.head()

,Refresh,Last Update : 2019-05-30 14:38:23,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 1188,Unnamed: 1189,Unnamed: 1190,Unnamed: 1191,Unnamed: 1192,Unnamed: 1193,Unnamed: 1194,Unnamed: 1195,Unnamed: 1196,Unnamed: 1197
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Time Series (Company),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Frequency,M,Ascending,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Period(From),19990101,Korean,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Period(To),CPD [20190530],NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
in_k200.head()

,Refresh,Last Update : 2019-05-30 14:39:36,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 1188,Unnamed: 1189,Unnamed: 1190,Unnamed: 1191,Unnamed: 1192,Unnamed: 1193,Unnamed: 1194,Unnamed: 1195,Unnamed: 1196,Unnamed: 1197
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Time Series (Company),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Frequency,M,Ascending,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Period(From),19990101,Korean,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Period(To),CPD [20190530],NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# iloc 

## 1) code를 다 불러오기
raw_per.iloc[6,1:]  # iloc : 행, 열의 순서로. 0부터 시작하는 셀의 위치로 값 불러오기
codeName = raw_per.iloc[6,1:].values  # 값만 불러오고 싶을때
codeName#
#
## 2) 날짜 다 불러오기
dateList = raw_per.iloc[13:,0].values
#
## 3) 값들을 다 불러오기
allValues = raw_per.iloc[13:,1:].values
#
#
## 4) dataframe 형식으로 새로 만들어주기
#'''
#열 : 코드
#행 : 날짜
#그 안에 값들..
#'''
new_per_data = pd.DataFrame(allValues, columns=codeName, index = dateList)


In [3]:
def data_cleansing_quantiwise(rawData):
    '''Quantiwise 제공 시계열데이터 클렌징 용도
    - 열 : 종목명
    - 행 : 시계열
    '''    
    firmCode = rawData.iloc[6, 1:].values
    dateIndex = rawData.iloc[13:, 0].values
    newData = rawData.iloc[13:,1:]
    newData.columns = firmCode
    newData.index = dateIndex
    return newData

new_per_data = data_cleansing_quantiwise(raw_per)

In [7]:
# 모든 열의 이름 (종목코드) 확인
new_per_data.columns

Index(['A000010', 'A000020', 'A000030', 'A000040', 'A000050', 'A000060',
       'A000070', 'A000080', 'A000090', 'A000100',
       ...
       'A300720', 'A306200', 'A307950', 'A316140', 'A900030', 'A900050',
       'A900140', 'A950010', 'A950070', 'A950100'],
      dtype='object', length=1197)

In [8]:
new_per_data.index

DatetimeIndex(['1999-01-29', '1999-02-26', '1999-03-31', '1999-04-30',
               '1999-05-31', '1999-06-30', '1999-07-30', '1999-08-31',
               '1999-09-30', '1999-10-29',
               ...
               '2018-08-31', '2018-09-28', '2018-10-31', '2018-11-30',
               '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-29',
               '2019-04-30', '2019-05-29'],
              dtype='datetime64[ns]', length=245, freq=None)

In [6]:
# 첫번째 날 확인
month_1 = new_per_data.index[0]
month_1

Timestamp('1999-01-29 00:00:00')

In [7]:
new_per_data[1::3].index  # 매 분기 두번째 달만 불러오기

# 1 -> 0부터 시작하기 때문에 인덱스의 두번째부터 불러오기 시작한다는 것
# : -> 끝까지
# :3 -> 3칸씩 띄어서 값을 불러오겠다는 것

DatetimeIndex(['1999-02-26', '1999-05-31', '1999-08-31', '1999-11-30',
               '2000-02-29', '2000-05-31', '2000-08-31', '2000-11-30',
               '2001-02-28', '2001-05-31', '2001-08-31', '2001-11-30',
               '2002-02-28', '2002-05-31', '2002-08-30', '2002-11-29',
               '2003-02-28', '2003-05-30', '2003-08-29', '2003-11-28',
               '2004-02-27', '2004-05-31', '2004-08-31', '2004-11-30',
               '2005-02-28', '2005-05-31', '2005-08-31', '2005-11-30',
               '2006-02-28', '2006-05-31', '2006-08-31', '2006-11-30',
               '2007-02-28', '2007-05-31', '2007-08-31', '2007-11-30',
               '2008-02-29', '2008-05-30', '2008-08-29', '2008-11-28',
               '2009-02-27', '2009-05-29', '2009-08-31', '2009-11-30',
               '2010-02-26', '2010-05-31', '2010-08-31', '2010-11-30',
               '2011-02-28', '2011-05-31', '2011-08-31', '2011-11-30',
               '2012-02-29', '2012-05-31', '2012-08-31', '2012-11-30',
      

In [9]:
rebalancing_schedule = new_per_data[1::3].index 

In [10]:
# loc : iloc과 유사, 행과 열의 순서로 셀값 불러오기. 다만 셀의 위치를 숫자가 아닌 인덱스의 이름으로.
# 해당 월의 PER 데이터 추출하기
per_1 = new_per_data.loc[rebalancing_schedule[8], :]

In [11]:
per_1

A000010     5.01
A000020    10.19
A000030     1.13
A000040      NaN
A000050      NaN
A000060     4.28
A000070      NaN
A000080      NaN
A000090      NaN
A000100     8.86
A000110      NaN
A000120      NaN
A000130      NaN
A000140     9.98
A000150     3.88
A000160      NaN
A000170      NaN
A000180      NaN
A000200      NaN
A000210     3.94
A000220      NaN
A000230      NaN
A000240     7.82
A000270    12.53
A000280      NaN
A000300      NaN
A000310      NaN
A000320      NaN
A000330      NaN
A000360     6.38
           ...  
A267850      NaN
A268280      NaN
A271560      NaN
A271980      NaN
A272450      NaN
A272550      NaN
A280360      NaN
A281820      NaN
A282330      NaN
A282690      NaN
A284740      NaN
A285130      NaN
A286940      NaN
A293480      NaN
A294870      NaN
A298000      NaN
A298020      NaN
A298040      NaN
A298050      NaN
A298690      NaN
A300720      NaN
A306200      NaN
A307950      NaN
A316140      NaN
A900030      NaN
A900050      NaN
A900140      NaN
A950010      N

In [53]:
per_1 >= 0

A000010     True
A000020     True
A000030     True
A000040    False
A000050    False
A000060     True
A000070    False
A000080    False
A000090    False
A000100     True
A000110    False
A000120    False
A000130    False
A000140     True
A000150     True
A000160    False
A000170    False
A000180    False
A000200    False
A000210     True
A000220    False
A000230    False
A000240     True
A000270     True
A000280    False
A000300    False
A000310    False
A000320    False
A000330    False
A000360     True
           ...  
A267850    False
A268280    False
A271560    False
A271980    False
A272450    False
A272550    False
A280360    False
A281820    False
A282330    False
A282690    False
A284740    False
A285130    False
A286940    False
A293480    False
A294870    False
A298000    False
A298020    False
A298040    False
A298050    False
A298690    False
A300720    False
A306200    False
A307950    False
A316140    False
A900030    False
A900050    False
A900140    False
A950010    Fal

In [12]:
per_1[per_1>=0]

A000010     5.01
A000020    10.19
A000030     1.13
A000060     4.28
A000100     8.86
A000140     9.98
A000150     3.88
A000210     3.94
A000240     7.82
A000270    12.53
A000360     6.38
A000500     2.47
A000640     6.35
A000660     11.4
A000700     1.24
A000720    55.51
A000810    11.96
A000830    10.93
A000880     4.43
A001040    10.69
A001120      3.5
A001230     9.88
A001300     4.27
A001370     6.41
A001440     1.27
A001450     3.18
A001680     4.11
A001740     2.74
A001800     3.02
A001830    17.67
           ...  
A020560     3.72
A023590    27.31
A024110     3.79
A025000     2.54
A025530     7.52
A025540     6.33
A025560    13.89
A025830     1.51
A025850     4.05
A025860     7.03
A025930     4.59
A026870     3.86
A028050     7.43
A029530     5.56
A030000    11.59
A030200    20.46
A030210     2.22
A031820     9.56
A032390    19.31
A032640    15.51
A033240     7.66
A033530     6.71
A033780     8.75
A034020     6.89
A034120    10.29
A035000     4.29
A035720     99.8
A036460     5.

In [13]:
per_1 = per_1[per_1>=0]

In [119]:
per_1 = per_1.dropna()   # dropna : na 가 들어간 셀을 지워줌
per_1

A000010     5.01
A000020    10.19
A000030     1.13
A000060     4.28
A000100     8.86
A000140     9.98
A000150     3.88
A000210     3.94
A000240     7.82
A000270    12.53
A000360     6.38
A000500     2.47
A000640     6.35
A000660     11.4
A000700     1.24
A000720    55.51
A000810    11.96
A000830    10.93
A000880     4.43
A001040    10.69
A001120      3.5
A001230     9.88
A001300     4.27
A001370     6.41
A001440     1.27
A001450     3.18
A001680     4.11
A001740     2.74
A001800     3.02
A001830    17.67
           ...  
A020560     3.72
A023590    27.31
A024110     3.79
A025000     2.54
A025530     7.52
A025540     6.33
A025560    13.89
A025830     1.51
A025850     4.05
A025860     7.03
A025930     4.59
A026870     3.86
A028050     7.43
A029530     5.56
A030000    11.59
A030200    20.46
A030210     2.22
A031820     9.56
A032390    19.31
A032640    15.51
A033240     7.66
A033530     6.71
A033780     8.75
A034020     6.89
A034120    10.29
A035000     4.29
A035720     99.8
A036460     5.

In [58]:
# 그 중에서 하위 30 종목
per_1.astype(float)  # 그전에 data type을 실수(float)로 바꾸기 -> 데이터 타입이 실수로 되어있어야 숫자끼리 비교 가능?

A000010     5.01
A000020    10.19
A000030     1.13
A000060     4.28
A000100     8.86
A000140     9.98
A000150     3.88
A000210     3.94
A000240     7.82
A000270    12.53
A000360     6.38
A000500     2.47
A000640     6.35
A000660    11.40
A000700     1.24
A000720    55.51
A000810    11.96
A000830    10.93
A000880     4.43
A001040    10.69
A001120     3.50
A001230     9.88
A001300     4.27
A001370     6.41
A001440     1.27
A001450     3.18
A001680     4.11
A001740     2.74
A001800     3.02
A001830    17.67
           ...  
A020560     3.72
A023590    27.31
A024110     3.79
A025000     2.54
A025530     7.52
A025540     6.33
A025560    13.89
A025830     1.51
A025850     4.05
A025860     7.03
A025930     4.59
A026870     3.86
A028050     7.43
A029530     5.56
A030000    11.59
A030200    20.46
A030210     2.22
A031820     9.56
A032390    19.31
A032640    15.51
A033240     7.66
A033530     6.71
A033780     8.75
A034020     6.89
A034120    10.29
A035000     4.29
A035720    99.80
A036460     5.

In [59]:
per_1.astype(float).nsmallest(30)  # 작은거 30개

A000030    1.13
A000700    1.24
A001440    1.27
A025830    1.51
A011200    1.88
A003030    2.10
A030210    2.22
A005300    2.24
A015360    2.33
A006360    2.36
A003690    2.37
A005950    2.43
A000500    2.47
A025000    2.54
A002300    2.65
A020000    2.67
A001740    2.74
A005850    2.77
A013580    2.79
A009760    2.85
A013570    2.91
A016450    2.91
A017940    2.97
A003300    3.00
A005440    3.01
A001800    3.02
A001450    3.18
A017300    3.18
A009720    3.19
A012630    3.24
Name: 2001-02-28 00:00:00, dtype: float64

In [60]:
per_1.astype(float).nsmallest(30).index  # 작은거 30개의 종목코드

Index(['A000030', 'A000700', 'A001440', 'A025830', 'A011200', 'A003030',
       'A030210', 'A005300', 'A015360', 'A006360', 'A003690', 'A005950',
       'A000500', 'A025000', 'A002300', 'A020000', 'A001740', 'A005850',
       'A013580', 'A009760', 'A013570', 'A016450', 'A017940', 'A003300',
       'A005440', 'A001800', 'A001450', 'A017300', 'A009720', 'A012630'],
      dtype='object')

In [19]:
df_code = pd.DataFrame(per_1.astype(float).nsmallest(30).index, columns = ['code'])

In [20]:
df_code

,code
0,A000030
1,A000700
2,A001440
3,A025830
4,A011200
5,A003030
6,A030210
7,A005300
8,A015360
9,A006360


In [21]:
df_code['date'] = rebalancing_schedule[8]
df_code

,code,date
0,A000030,2001-02-28
1,A000700,2001-02-28
2,A001440,2001-02-28
3,A025830,2001-02-28
4,A011200,2001-02-28
5,A003030,2001-02-28
6,A030210,2001-02-28
7,A005300,2001-02-28
8,A015360,2001-02-28
9,A006360,2001-02-28


In [22]:
np.ones(5) / 5

array([0.2, 0.2, 0.2, 0.2, 0.2])

In [23]:
np.ones(len(df_code))  / len(df_code)

array([0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333])

In [24]:
df_code['weight'] = np.ones(len(df_code)) / len(df_code)
df_code

,code,date,weight
0,A000030,2001-02-28,0.033333
1,A000700,2001-02-28,0.033333
2,A001440,2001-02-28,0.033333
3,A025830,2001-02-28,0.033333
4,A011200,2001-02-28,0.033333
5,A003030,2001-02-28,0.033333
6,A030210,2001-02-28,0.033333
7,A005300,2001-02-28,0.033333
8,A015360,2001-02-28,0.033333
9,A006360,2001-02-28,0.033333


In [69]:
df_code = df_code[['date', 'code', 'weight']]
df_code

,date,code,weight
0,2001-02-28,A000030,0.033333
1,2001-02-28,A000700,0.033333
2,2001-02-28,A001440,0.033333
3,2001-02-28,A025830,0.033333
4,2001-02-28,A011200,0.033333
5,2001-02-28,A003030,0.033333
6,2001-02-28,A030210,0.033333
7,2001-02-28,A005300,0.033333
8,2001-02-28,A015360,0.033333
9,2001-02-28,A006360,0.033333


## Week 4 : 전체 시계열로 확장하기 (For loop, Concatenate)

### Considerations
- Issue 1. Outliers --> Z-score & Winsorize
- Issue 2. The number of constituents --> "p-hacking" problem

In [65]:
# Issue 1 Only. 
# 1) Z-score 
(per_1 - per_1.mean()) / per_1.std()

A000010    -0.323081
A000020     0.209536
A000030     -0.72203
A000060    -0.398141
A000100    0.0727827
A000140     0.187943
A000150     -0.43927
A000210      -0.4331
A000240    -0.034152
A000270     0.450139
A000360    -0.182215
A000500    -0.584248
A000640      -0.1853
A000660      0.33395
A000700    -0.710719
A000720      4.86942
A000810      0.39153
A000830     0.285624
A000880    -0.382718
A001040     0.260947
A001120    -0.478342
A001230     0.177661
A001300    -0.399169
A001370    -0.179131
A001440    -0.707635
A001450    -0.511245
A001680    -0.415621
A001740    -0.556487
A001800    -0.527696
A001830     0.978643
             ...    
A020560    -0.455721
A023590      1.96984
A024110    -0.448524
A025000    -0.577051
A025530   -0.0649985
A025540    -0.187356
A025560     0.589976
A025830    -0.682957
A025850     -0.42179
A025860    -0.115381
A025930    -0.366266
A026870    -0.441326
A028050   -0.0742525
A029530    -0.266529
A030000     0.353486
A030200      1.26552
A030210    -0

In [24]:
def to_zscore(factor):
    return (factor - factor.mean()) / factor.std()

1.1449174941446927e-14

In [94]:
from scipy import stats
stats.mstats.winsorize(per_1, limits=[0.1, 0.1])

In [122]:
np.mean(per_1)

8.152147239263803

In [123]:
np.mean(stats.mstats.winsorize(per_1, limits=[0.1, 0.1]))

6.7674846625766865